In [1]:
include("../src/diffusionsine.jl")

diffusionsine (generic function with 1 method)

In [6]:
#srand(128)
nsteps = 1000
nsampling = 1000
dt = 1e-4
Dx = Dy = 1.
lambda = 0.1
sigma =1.0
nparticles = 1;

In [7]:
positions = diffusionsine(nparticles, nsteps, nsampling, dt, Dx,Dy, lambda, sigma);

Particle 1 done


In [9]:
positions = zeros(nsteps,2, nparticles);

In [103]:
shape(s) = x->sin(s*x)
b = Boundary(shape(sigma), lambda)

Boundary(s,t,n,0.1)

In [257]:
n = 1000
positions = zeros(n,2, nparticles);
srand(128)
nsteps = 812
j = 1
p = Particle([0.,0.89], [0.,0.])
positions[1,:,1] = p.r
 for i in 2:nsteps
    p.rprevious = positions[i-1,:,j]
    p.r += sqrt(2*dt)*[Dx*randn(), Dy*randn()]
    boundarysine(p, b, sigma)
    boundary1(p, b)
    positions[i,:,j] = p.r 
end
println("Particle $j done")

i = 813
p.rprevious = positions[i-1,:,j]
p.r += sqrt(2*dt)*[Dx*randn(), Dy*randn()]
intersection(x) = b.shape(x) - straightlineparticle(p)(x)
xint = fzero(intersection,[p.r[1], p.rprevious[1]])
yint = b.shape(xint)
rint = p.r - [xint, yint]
tanvector = dot(rint, b.tangent(xint))*b.tangent(xint)
norvector = -dot(rint, b.normal(xint))*b.normal(xint) ##Minus to change the sign of the normal vector
p.r = [xint, yint] + tanvector + norvector

if !incell(p,b)
    p.r = p.rprevious
end

boundary1(p, b)
positions[i,:,j] = p.r 

i = 814

p.rprevious = positions[i-1,:,j]
p.r += sqrt(2*dt)*[Dx*randn(), Dy*randn()]
boundarysine(p, b, sigma)
boundary1(p, b)
positions[i,:,j] = p.r 

#boundarysine(p, b, sigma)
#boundary1(p, b)
#positions[i,:,j] = p.r 

Particle 1 done


2-element Array{Float64,1}:
 4.6605  
 0.800972

In [258]:
p

Particle{Float64}([4.6605,0.800972],[4.6985,0.916543])

In [259]:
1.0+lambda*sin(sigma*p.r[1])

1.0495906648136821

In [4]:
using PyPlot

In [8]:
pygui(true)
plot(positions[:,:,1][:,1],positions[:,:,1][:,2],".-")
x = minimum(positions[:,:,1][:,1]):0.01:maximum(positions[:,:,1][:,1])
#plot(p.r, "go")
plot(x, 1.0+lambda*sin(sigma*x))

1-element Array{Any,1}:
 PyObject <matplotlib.lines.Line2D object at 0x7f0d64832b50>

In [28]:
positions[:,:,1][:,1],positions[:,:,1][:,2]

([0.0,-0.639752,-0.819665,-0.930948,-1.98975,-1.57294,-2.0395,-2.33916,-2.62219,-2.86883  …  6.09119,5.76268,5.4515,6.26805,6.29052,5.98278,6.53843,6.08437,6.3952,6.23242],[0.0,0.732923,0.51086,0.89269,0.776381,0.213609,0.0167583,0.756812,0.145497,0.759956  …  0.314127,0.405808,0.525351,0.960427,0.489512,0.367382,0.582788,0.417574,0.345852,0.847158])

In [13]:
#plot([0.180986,0.182268],[1.39155,1.3873], "o-")
pygui(true)
plot([-1.41975,-1.40953],[0.900163,0.900082],"go-")
x = -5:0.0001:5
plot(x, 1.0+lambda*sin(sigma*x))

1-element Array{Any,1}:
 PyObject <matplotlib.lines.Line2D object at 0x7ff8bd98a550>

In [31]:
xarray = positions[:,1,:]
xsquare = [mean(xarray[i,:].^2) for i in 1:length(xarray[:,1])]
#xmean =  [mean(xarray[i,:]) for i in 1:length(xarray[:,1])]
#crec = xsquare - xmean.^2
t = 0.0:nsampling*dt:dt*nsampling*(nsteps-1)
plot(collect(t),xsquare./(2*Dx*t))

1000-element Array{Float64,1}:
  0.0     
 -0.127253
 -0.375259
 -0.245432
 -0.415052
 -0.228413
 -0.433109
 -0.443209
 -0.411467
 -0.536935
 -0.476811
 -0.516892
 -0.714509
  ⋮       
 -3.38142 
 -3.51367 
 -3.58562 
 -3.81245 
 -3.7577  
 -3.68155 
 -3.71623 
 -3.98388 
 -3.78552 
 -3.90173 
 -3.98215 
 -4.33909 

In [32]:
plot(xmean)

1-element Array{Any,1}:
 PyObject <matplotlib.lines.Line2D object at 0x7f901fff5650>

0.0:0.1:99.9

In [9]:
t = 0.0:nsampling*dt:dt*nsampling*(nsteps-1)

0.0:0.01:99.99000000000001

In [46]:
collect(t)

1000-element Array{Float64,1}:
  0.0
  0.1
  0.2
  0.3
  0.4
  0.5
  0.6
  0.7
  0.8
  0.9
  1.0
  1.1
  1.2
  ⋮  
 98.8
 98.9
 99.0
 99.1
 99.2
 99.3
 99.4
 99.5
 99.6
 99.7
 99.8
 99.9

In [48]:
plot(collect(t),crec./(2*Dx*t))

1-element Array{Any,1}:
 PyObject <matplotlib.lines.Line2D object at 0x7f901f207050>

In [17]:
p = Particle([4.53653,1.1017],[4.55757,1.09997])

Particle{Float64}([4.53653,1.1017],[4.55757,1.09997])

In [18]:
shape(s) = x->sin(s*x)
b = Boundary(shape(sigma), lambda)

Boundary(s,t,n,0.1)

In [28]:
intersection(x) = b.shape(x) - straightlineparticle(p)(x)
xint = fzero(intersection,p.r[1])
yint = b.shape(xint)
 tanvector = dot(rint, b.tangent(xint))*b.tangent(xint)
    norvector = -dot(rint, b.normal(xint))*b.normal(xint) ##Minus to change the sign of the normal vector
    p.rprevious = [xint, yint]
    p.r = [xint, yint] + tanvector + norvector


2-element Array{Float64,1}:
 4.52522
 1.09373

In [13]:
reflectingboundaries!(p,b)

2-element Array{Float64,1}:
 4.53653
 1.1017 

In [29]:
p

Particle{Float64}([4.52522,1.09373],[4.54545,1.09951])

In [31]:
#plot([0.180986,0.182268],[1.39155,1.3873], "o-")
pygui(true)
plot([4.53653,4.55757],[1.1017,1.09997],"go-")
plot([4.53647, 4.5575],[1.09917,1.09998], "bo-")
plot([4.52522,4.54545],[1.09373,1.09951], "ro-")
#plot([4.53653,4.5575],[1.1017,1.09998],"ro-")
x = -5:0.0001:5
plot(x, 1.0+lambda*sin(sigma*x) )

1-element Array{Any,1}:
 PyObject <matplotlib.lines.Line2D object at 0x7f6b48f1d110>